In [1]:
import torch
import torch.nn as nn
import logging
import h5py # Save - Load 3D tensor
import numpy as np
import torch.optim as optim
import optuna
from tqdm import tqdm 

import torch.nn.init as init
import torch.nn.functional as F

import matplotlib.pyplot as plt
# display Matplotlib plots directly within the notebook interface
%matplotlib inline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_squared_log_error
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("logging.log"),
        logging.StreamHandler()
    ]
)

In [3]:
file_name = 'hierarchical_attempt_II'

In [4]:
# Load tensors from the HDF5 file
load_path = 'CSV/exports/tensors/o4_3D_four_dataframe_hierarchical.h5'

logging.info(f"Loading...")
with h5py.File(load_path, 'r') as hf:
    train_tensor = hf['train_tensor'][:]
    validate_tensor = hf['validate_tensor'][:]
    test_tensor = hf['test_tensor'][:]
    external_tensor = hf['external_tensor'][:]
    # los
    train_los_label = hf['train_los_label'][:]
    validate_los_label = hf['validate_los_label'][:]
    test_los_label = hf['test_los_label'][:]
    external_los_label = hf['external_los_label'][:]
    # mortality
    train_mortality_label = hf['train_mortality_label'][:]
    validate_mortality_label = hf['validate_mortality_label'][:]
    test_mortality_label = hf['test_mortality_label'][:]
    external_mortality_label = hf['external_mortality_label'][:]

logging.info(f"Train: {train_tensor.shape}, Los Label: {train_los_label.shape}, Mortality Label: {train_mortality_label.shape}")
logging.info(f"Validate: {validate_tensor.shape}, Los Label: {validate_los_label.shape}, Mortality Label: {validate_mortality_label.shape}")
logging.info(f"Test: {test_tensor.shape}, Los Label: {test_los_label.shape}, Mortality Label: {test_mortality_label.shape}")
logging.info(f"External: {external_tensor.shape}, Los Label: {external_los_label.shape}, Mortality Label: {external_mortality_label.shape}")

2025-02-22 11:42:34,451 - INFO - Loading...
2025-02-22 11:42:35,466 - INFO - Train: (122496, 345, 4), Los Label: (122496, 1), Mortality Label: (122496, 1)
2025-02-22 11:42:35,467 - INFO - Validate: (15312, 345, 4), Los Label: (15312, 1), Mortality Label: (15312, 1)
2025-02-22 11:42:35,468 - INFO - Test: (15312, 345, 4), Los Label: (15312, 1), Mortality Label: (15312, 1)
2025-02-22 11:42:35,470 - INFO - External: (234720, 345, 4), Los Label: (234720, 1), Mortality Label: (234720, 1)


In [5]:
class LSTMAttentionMultiTask(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_layers, dropout):
        super(LSTMAttentionMultiTask, self).__init__()
        self.hidden_sizes = hidden_sizes
        self.num_layers = num_layers

        # Define LSTM layers dynamically
        self.lstms = nn.ModuleList()
        for i in range(num_layers):
            input_dim = input_size if i == 0 else hidden_sizes[i-1]
            hidden_dim = hidden_sizes[i]

            if num_layers > 1:
                self.lstms.append(nn.LSTM(input_dim, hidden_dim, batch_first=True, dropout=dropout))
            else:
                self.lstms.append(nn.LSTM(input_dim, hidden_dim, batch_first=True))  # No dropout when num_layers=1

        # Attention Layer
        self.attn = nn.Linear(hidden_sizes[-1], 1)

        # Fully Connected Layers for Multi-task Learning
        self.fc_los = nn.Linear(hidden_sizes[-1], 1)  # Regression Output (LOS)
        self.fc_mortality = nn.Linear(hidden_sizes[-1], 1)  # Classification Output (Mortality)

    def forward(self, x):
        batch_size = x.size(0)
        out = x

        # Initialize hidden and cell states dynamically for each LSTM layer
        hidden_states = [(torch.zeros(1, batch_size, h).to(x.device), torch.zeros(1, batch_size, h).to(x.device)) 
                         for h in self.hidden_sizes]

        for i, lstm in enumerate(self.lstms):
            out, hidden_states[i] = lstm(out, hidden_states[i])

        # Attention Mechanism
        attn_weights = torch.tanh(self.attn(out))
        attn_weights = torch.softmax(attn_weights, dim=1)
        context_vector = torch.sum(attn_weights * out, dim=1)

        # Multi-task outputs
        los_output = self.fc_los(context_vector)  # Regression
        mortality_output = torch.sigmoid(self.fc_mortality(context_vector))  # Classification

        return los_output, mortality_output, attn_weights

In [6]:
# Load data
X_train = torch.tensor(train_tensor, dtype=torch.float32)
y_train = torch.tensor(train_los_label, dtype=torch.float32)

X_validate = torch.tensor(validate_tensor, dtype=torch.float32)
y_validate = torch.tensor(validate_los_label, dtype=torch.float32)

In [8]:
# Load tensors from the HDF5 file
load_path = 'CSV/exports/tensors/o4_3D_four_dataframe_hierarchical.h5'
logging.info("Loading tensors...")
with h5py.File(load_path, 'r') as hf:
    train_tensor = hf['train_tensor'][:]
    train_los_label = hf['train_los_label'][:]
    train_mortality_label = hf['train_mortality_label'][:]
# Create a balanced 10% sample from the training data
def create_stratified_sample(data, labels, sample_size=0.1):
    # Find indices for each class
    non_survivor_indices = np.where(labels == 1)[0]  # Non-survivors
    survivor_indices = np.where(labels == 0)[0]      # Survivors

    # Determine number of samples from each class
    n_non_survivors = int(len(non_survivor_indices) * sample_size)
    n_survivors = int(len(survivor_indices) * sample_size)

    # Randomly select indices for a balanced dataset
    sampled_non_survivors_indices = np.random.choice(non_survivor_indices, n_non_survivors, replace=False)
    sampled_survivors_indices = np.random.choice(survivor_indices, n_survivors, replace=False)
    sampled_indices = np.concatenate([sampled_non_survivors_indices, sampled_survivors_indices])

    return sampled_indices

# Usage of the function to sample data
sampled_indices = create_stratified_sample(train_tensor, train_mortality_label)
sampled_train_tensor = train_tensor[sampled_indices]
sampled_train_los_label = train_los_label[sampled_indices]
sampled_train_mortality_label = train_mortality_label[sampled_indices]
logging.info("Load ends")

2025-02-22 11:43:04,643 - INFO - Loading tensors...
2025-02-22 11:43:05,073 - INFO - Load ends...


In [9]:
# Define the model class (modify according to your actual model implementation)
class LSTMAttentionMultiTask(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_layers, dropout):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_sizes[0], num_layers=num_layers, dropout=dropout, batch_first=True)
        self.fc1 = nn.Linear(hidden_sizes[0], 1)  # Output for LOS
        self.fc2 = nn.Linear(hidden_sizes[0], 1)  # Output for mortality

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out_los = self.fc1(lstm_out[:, -1])
        out_mortality = torch.sigmoid(self.fc2(lstm_out[:, -1]))
        return out_los, out_mortality

# Objective function for Optuna
def objective(trial):
    num_layers = trial.suggest_int("num_layers", 1, 3)
    hidden_sizes = [trial.suggest_int(f"hidden_size_{i}", 64, 256, step=64) for i in range(num_layers)]
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    
    if num_layers == 1:
        dropout = 0.0

    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    model = LSTMAttentionMultiTask(input_size=4, hidden_sizes=hidden_sizes, num_layers=num_layers, dropout=dropout)
    criterion_los = nn.MSELoss()
    criterion_mortality = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Simulated train_loader (replace with actual DataLoader)
    train_loader = DataLoader(TensorDataset(torch.tensor(sampled_train_tensor, dtype=torch.float32),
                                            torch.tensor(sampled_train_los_label, dtype=torch.float32),
                                            torch.tensor(sampled_train_mortality_label, dtype=torch.float32)),
                              batch_size=32, shuffle=True)

    # Training loop with progress bar
    for epoch in range(3):  # Reduced number of epochs for simplicity
        model.train()
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{3}", leave=True)  # Progress bar setup
        for X_batch, y_los_batch, y_mort_batch in progress_bar:
            pred_los, pred_mortality = model(X_batch)
            loss_los = criterion_los(pred_los.squeeze(), y_los_batch.squeeze())
            loss_mortality = criterion_mortality(pred_mortality.squeeze(), y_mort_batch.squeeze())
            loss = loss_los + loss_mortality
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            progress_bar.set_postfix({"Loss": total_loss / len(train_loader)})  # Update progress bar with loss info

    return total_loss / len(train_loader)

# Run Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)  # Reduced number of trials for simplicity

# Log best hyperparameters
best_params = study.best_params
logging.info(f"Best Hyperparameters: {best_params}")

[I 2025-02-22 11:43:12,689] A new study created in memory with name: no-name-2aa8afbf-01a2-4a2b-a045-ebabeb782eae
Epoch 3/3: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 383/383 [15:48<00:00,  2.48s/it, Loss=5.83]
[I 2025-02-22 12:23:18,792] Trial 0 finished with value: 5.8292386519379775 and parameters: {'num_layers': 2, 'hidden_size_0': 256, 'hidden_size_1': 256, 'dropout': 0.24931011617233784, 'lr': 0.0017358883046590327}. Best is trial 0 with value: 5.8292386519379775.
Epoch 3/3: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 383/383 [03:38<00:00,  1.75it/s, Loss=5.84]
[I 2025-02-22 12:34:04,969] Trial 1 finished with value: 5.8398036483684965 and parameters: {'num_layers': 3, 'hidden_size_0': 128, 'hidden_size_1': 192, 'hidden_size_2': 192, 'dropout': 0.4265309812916264, 'lr': 0.0023066662107398017}.

In [11]:
# Load tensors from the HDF5 file
load_path = 'CSV/exports/tensors/o4_3D_four_dataframe_hierarchical.h5'
logging.info("Loading tensors...")

with h5py.File(load_path, 'r') as hf:
    train_tensor = hf['train_tensor'][:]
    train_los_label = hf['train_los_label'][:]
    train_mortality_label = hf['train_mortality_label'][:]

logging.info(f"Train: {train_tensor.shape}, Los Label: {train_los_label.shape}, Mortality Label: {train_mortality_label.shape}")

# Convert numpy arrays to torch tensors
train_tensor = torch.tensor(train_tensor, dtype=torch.float32)
train_los_label = torch.tensor(train_los_label, dtype=torch.float32)
train_mortality_label = torch.tensor(train_mortality_label, dtype=torch.float32)

# Setup the DataLoader for the training data
train_dataset = TensorDataset(train_tensor, train_los_label, train_mortality_label)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

best_hidden_sizes = [best_params[f"hidden_size_{i}"] for i in range(best_params["num_layers"])]
best_dropout = best_params["dropout"]
best_lr = best_params["lr"]

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model
import torch
from torch import nn

class LSTMAttentionMultiTask(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_layers, dropout):
        super(LSTMAttentionMultiTask, self).__init__()
        self.hidden_sizes = hidden_sizes
        self.num_layers = num_layers
        self.lstms = nn.ModuleList()
        for i in range(num_layers):
            input_dim = input_size if i == 0 else hidden_sizes[i-1]
            hidden_dim = hidden_sizes[i]
            if i < num_layers - 1:
                self.lstms.append(nn.LSTM(input_dim, hidden_dim, batch_first=True, dropout=dropout))
            else:
                self.lstms.append(nn.LSTM(input_dim, hidden_dim, batch_first=True))

        self.attn = nn.Linear(hidden_sizes[-1], 1)
        self.fc_los = nn.Linear(hidden_sizes[-1], 1)
        self.fc_mortality = nn.Linear(hidden_sizes[-1], 1)

    def forward(self, x):
        batch_size = x.size(0)
        out = x
        hidden_states = [(torch.zeros(1, batch_size, h).to(x.device), torch.zeros(1, batch_size, h).to(x.device)) for h in self.hidden_sizes]
        for i, lstm in enumerate(self.lstms):
            out, hidden_states[i] = lstm(out, hidden_states[i])
        attn_weights = torch.tanh(self.attn(out))
        attn_weights = torch.softmax(attn_weights, dim=1)
        context_vector = torch.sum(attn_weights * out, dim=1)
        los_output = self.fc_los(context_vector)
        mortality_output = torch.sigmoid(self.fc_mortality(context_vector))
        return los_output, mortality_output


# Instantiate and move the model to the appropriate device
best_model = LSTMAttentionMultiTask(input_size=4, hidden_sizes=best_hidden_sizes, num_layers=best_params["num_layers"], dropout=best_dropout).to(device)

# Loss functions and optimizer
criterion_los = nn.MSELoss()
criterion_mortality = nn.BCELoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_lr)

# Training loop
epochs = 10
for epoch in range(epochs):
    best_model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}", leave=True)
    for X_batch, y_los_batch, y_mort_batch in progress_bar:
        X_batch, y_los_batch, y_mort_batch = X_batch.to(device), y_los_batch.to(device), y_mort_batch.to(device)
        optimizer.zero_grad()
        pred_los, pred_mortality = best_model(X_batch)
        loss_los = criterion_los(pred_los.squeeze(), y_los_batch.squeeze())
        loss_mortality = criterion_mortality(pred_mortality.squeeze(), y_mort_batch.squeeze())
        loss = loss_los + loss_mortality
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        progress_bar.set_postfix({"Loss": total_loss / len(train_loader)})

logging.info("Training complete.")
torch.save(best_model.state_dict(), "models/final_lstm_attention_model.pth")
logging.info("Model saved successfully.")

2025-02-22 15:00:54,750 - INFO - Loading tensors...
2025-02-22 15:00:55,067 - INFO - Train: (122496, 345, 4), Los Label: (122496, 1), Mortality Label: (122496, 1)
Epoch 6/10:   3%|███▌                                                                                                                           | 107/3828 [01:02<36:00,  1.72it/s, Loss=0.0954]


KeyboardInterrupt: 

In [ ]:
# Define model parameters
input_size = X_train.shape[2]  # Number of features
hidden_sizes = [256, 128, 64]  # LSTM layers sizes

# Instantiate model
model = LSTMAttentionMultiTask(input_size, hidden_sizes)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Print model architecture
logging.info(model)

In [ ]:
# Define Losses & Optimizer
criterion_los = nn.MSELoss()  # Loss for LOS (Regression)
criterion_mortality = nn.BCELoss()  # Loss for Mortality (Binary Classification)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

# Training Setup
epochs = 50
batch_size = 32
patience = 5
min_delta = 0.001
best_val_loss = float('inf')
patience_counter = 0
early_stop = False

# DataLoaders
train_dataset = TensorDataset(torch.tensor(train_tensor, dtype=torch.float32), 
                              torch.tensor(train_los_label, dtype=torch.float32),
                              torch.tensor(train_mortality_label, dtype=torch.float32))

validate_dataset = TensorDataset(torch.tensor(validate_tensor, dtype=torch.float32), 
                                 torch.tensor(validate_los_label, dtype=torch.float32),
                                 torch.tensor(validate_mortality_label, dtype=torch.float32))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
validate_loader = DataLoader(validate_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
# Move model to CPU before saving
model.to("cpu")

# Define the model save path
model_save_path = f"models/{file_name}_model.pth"

# Save the trained model's state dictionary
torch.save(model.state_dict(), model_save_path)

# Confirm model saving
print(f"Model saved successfully at {model_save_path}")

In [ ]:
"""Step 1: Get Attention Scores for a Sample"""
# Select a random sample from the validation set
sample_idx = np.random.randint(len(validate_dataset))
X_sample, _, _ = validate_dataset[sample_idx]  # Only take the input tensor
X_sample = X_sample.unsqueeze(0).to(device)  # Add batch dimension

# Get model output and attention weights
model.eval()
with torch.no_grad():
    _, _, attn_weights = model(X_sample)  # Get attention scores

# Convert attention weights to numpy
attn_weights = attn_weights.squeeze().cpu().numpy()  # Shape: (seq_len, 1)
seq_len = attn_weights.shape[0]

# Plot attention weights
plt.figure(figsize=(12, 5))
plt.plot(range(seq_len), attn_weights, marker="o", linestyle="-", label="Attention Score")
plt.xlabel("Time Step")
plt.ylabel("Attention Score")
plt.title("Attention Weights Over Time for a Single Patient")
plt.legend()
plt.savefig(f'plots/tensor/02_attention_weight/{file_name}_single_patient.png')
plt.show()

In [ ]:
"""Step 2: Compare Attention Across Multiple Patients"""
num_samples = 5  # Number of patients to visualize
all_attn_weights = []

# Select multiple random samples
random_indices = np.random.choice(len(validate_dataset), num_samples, replace=False)

for idx in random_indices:
    X_sample, _, _ = validate_dataset[idx]
    X_sample = X_sample.unsqueeze(0).to(device)  # Add batch dimension

    model.eval()
    with torch.no_grad():
        _, _, attn_weights = model(X_sample)

    attn_weights = attn_weights.squeeze().cpu().numpy()
    all_attn_weights.append(attn_weights)

# Convert to NumPy array and compute mean attention weights
all_attn_weights = np.array(all_attn_weights)  # Shape: (num_samples, seq_len)
mean_attn_weights = np.mean(all_attn_weights, axis=0)  # Average over patients

# Plot Mean Attention Weights
plt.figure(figsize=(12, 5))
plt.plot(range(seq_len), mean_attn_weights, marker="o", linestyle="-", color='red', label="Mean Attention Score")
plt.xlabel("Time Step")
plt.ylabel("Mean Attention Score")
plt.title("Average Attention Weights Over Multiple Patients")
plt.legend()
plt.savefig(f'plots/tensor/02_attention_weight/{file_name}_multiple_patients.png')
plt.show()

In [ ]:
"""Step 3: Visualize LOS vs. Mortality Attention Differences"""

num_samples = 5  # Number of patients to visualize
los_attn_weights = []
mortality_attn_weights = []

# Select multiple random samples
random_indices = np.random.choice(len(validate_dataset), num_samples, replace=False)

for idx in random_indices:
    X_sample, _, _ = validate_dataset[idx]
    X_sample = X_sample.unsqueeze(0).to(device)  # Add batch dimension

    model.eval()
    with torch.no_grad():
        los_pred, mort_pred, attn_weights = model(X_sample)

    attn_weights = attn_weights.squeeze().cpu().numpy()

    # Separate LOS and Mortality predictions' attention scores
    los_attn_weights.append(attn_weights * los_pred.item())  # Weight by LOS
    mortality_attn_weights.append(attn_weights * mort_pred.item())  # Weight by Mortality

# Compute mean attention scores
los_attn_weights = np.mean(np.array(los_attn_weights), axis=0)
mortality_attn_weights = np.mean(np.array(mortality_attn_weights), axis=0)

# Plot both attention weights
plt.figure(figsize=(12, 5))
plt.plot(range(seq_len), los_attn_weights, marker="o", linestyle="-", label="LOS Attention", color="blue")
plt.plot(range(seq_len), mortality_attn_weights, marker="o", linestyle="-", label="Mortality Attention", color="green")
plt.xlabel("Time Step")
plt.ylabel("Weighted Attention Score")
plt.title("Comparison of Attention Weights for LOS vs. Mortality")
plt.savefig(f'plots/tensor/02_attention_weight/{file_name}_los_vs_mortality.png')
plt.legend()
plt.show()

# Training Performance Plot

In [ ]:
# Ensure train_losses and val_losses are aligned
min_len = min(len(train_losses), len(val_losses))  # Align lengths if different
train_losses = train_losses[:min_len]
val_losses = val_losses[:min_len]

# Identify the best epoch where early stopping occurred
best_epoch = min_len - patience_counter  # patience_counter tracks epochs without improvement

# Plot Training Loss
fig, ax1 = plt.subplots(figsize=(10, 6))  # Initialize plot with size
line1 = ax1.plot(range(1, min_len + 1), train_losses, label='Training Loss', color='b')
ax1.set_xlabel('Epochs')  # Label for X-axis
ax1.set_ylabel('Training Loss', color='b')  # Label for Y-axis on the left
ax1.tick_params(axis='y', labelcolor='b')  # Left Y-axis tick color
ax1.grid(visible=True, linestyle='--', alpha=0.6)  # Add grid for clarity

# Plot Validation Loss on a Secondary Y-Axis
ax2 = ax1.twinx()  # Create twin axes for validation loss
line2 = ax2.plot(range(1, min_len + 1), val_losses, label='Validation Loss', color='orange')
ax2.set_ylabel('Validation Loss', color='orange')  # Label for Y-axis on the right
ax2.tick_params(axis='y', labelcolor='orange')  # Right Y-axis tick color

# Highlight Early Stopping Point
line3 = ax1.axvline(best_epoch, color='r', linestyle='--', label='Early Stopping Point')

# Combine Legends from Both Axes
lines = line1 + line2 + [line3]  # Combine lines from both Y-axes
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc='upper right')  # Display legend

# Add Title and Final Touches
plt.title('Training and Validation Loss Over Epochs with Early Stopping')
fig.tight_layout()  # Adjust spacing to prevent overlap
plt.savefig(f'plots/tensor/01_train_vall_loss/{file_name}_train_val_over_epoch.png')
# Display the Plot
plt.show()

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Ensure external_tensor and external_los_label are PyTorch tensors
if isinstance(external_tensor, np.ndarray):
    external_tensor = torch.tensor(external_tensor, dtype=torch.float32)
if isinstance(external_los_label, np.ndarray):
    external_los_label = torch.tensor(external_los_label, dtype=torch.float32)

# Print shapes to debug
print(f"external_tensor shape: {external_tensor.shape}")
print(f"external_los_label shape: {external_los_label.shape}")

# Check if dimensions match
if external_tensor.shape[0] != external_los_label.shape[0]:
    raise ValueError(f"Mismatch: external_tensor has {external_tensor.shape[0]} rows, "
                     f"but external_los_label has {external_los_label.shape[0]} rows.")

# Create DataLoader for External Validation Set
external_dataset = TensorDataset(external_tensor, external_los_label)
external_loader = DataLoader(external_dataset, batch_size=512, shuffle=False)

print("External DataLoader created successfully!")


import torch
from torch.utils.data import DataLoader, TensorDataset

# Ensure external_tensor and external_los_label are PyTorch tensors
if isinstance(external_tensor, np.ndarray):
    external_tensor = torch.tensor(external_tensor, dtype=torch.float32)
if isinstance(external_los_label, np.ndarray):
    external_los_label = torch.tensor(external_los_label, dtype=torch.float32)

# Reshape external_los_label to match the batch size
external_los_label = external_los_label.view(-1)  # Flatten to (234720,)

# Print shapes to debug
print(f"Final: external_tensor shape: {external_tensor.shape}")  # (234720, 345, 4)
print(f"Final: external_los_label shape: {external_los_label.shape}")  # (234720,)

# Create DataLoader for External Validation Set
external_dataset = TensorDataset(external_tensor, external_los_label)
external_loader = DataLoader(external_dataset, batch_size=512, shuffle=False)

print("External DataLoader created successfully!")

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import logging

# Create DataLoader for Test Set
test_dataset = TensorDataset(test_tensor, test_los_label)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

# Create DataLoader for External Validation Set
external_dataset = TensorDataset(external_tensor, external_los_label)
external_loader = DataLoader(external_dataset, batch_size=512, shuffle=False)

# Move model to evaluation mode
model.eval()

# Initialize lists to store predictions and true labels
y_test_preds, y_test_trues = [], []
y_external_preds, y_external_trues = [], []

# Run batch-wise inference for test set with progress bar
logging.info("Running inference on the Test Set...")
with torch.no_grad():
    for X_batch, y_batch in tqdm(test_loader, desc="Processing Test Set", unit="batch"):
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        y_pred_batch, _, _ = model(X_batch)  # Get predictions

        # Store predictions and labels
        y_test_preds.append(y_pred_batch.cpu().numpy())
        y_test_trues.append(y_batch.cpu().numpy())

# Run batch-wise inference for external validation set with progress bar
logging.info("Running inference on the External Validation Set...")
with torch.no_grad():
    for X_batch, y_batch in tqdm(external_loader, desc="Processing External Set", unit="batch"):
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        y_pred_batch, _, _ = model(X_batch)  # Get predictions

        # Store predictions and labels
        y_external_preds.append(y_pred_batch.cpu().numpy())
        y_external_trues.append(y_batch.cpu().numpy())

# Convert lists to NumPy arrays
y_test_true = np.concatenate(y_test_trues).squeeze()
y_test_pred = np.concatenate(y_test_preds).squeeze()

y_external_true = np.concatenate(y_external_trues).squeeze()
y_external_pred = np.concatenate(y_external_preds).squeeze()

# Calculate metrics for the Test Set
logging.info("Calculating metrics for the Test Set...")
test_mse = mean_squared_error(y_test_true, y_test_pred)
test_mae = mean_absolute_error(y_test_true, y_test_pred)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(y_test_true, y_test_pred) * 100  # R² in percentage

# Calculate metrics for the External Validation Set
logging.info("Calculating metrics for the External Validation Set...")
external_mse = mean_squared_error(y_external_true, y_external_pred)
external_mae = mean_absolute_error(y_external_true, y_external_pred)
external_rmse = np.sqrt(external_mse)
external_r2 = r2_score(y_external_true, y_external_pred) * 100  # R² in percentage

# Print results with progress messages
logging.info("Final Results:")
logging.info(f"Test Set - MSE: {test_mse:.2f}, MAE: {test_mae:.2f}, RMSE: {test_rmse:.2f}, R2: {test_r2:.2f}%")
logging.info(f"External Validation - MSE: {external_mse:.2f}, MAE: {external_mae:.2f}, RMSE: {external_rmse:.2f}, R2: {external_r2:.2f}%")

In [ ]:
# Logging info
logging.info("Calculating metrics for the Test Set...")

# Initialize error metrics
error_metrics = ['MSE', 'MAE', 'RMSE']
values = [test_mse, test_mae, test_rmse]

# Try to calculate MSLE
try:
    test_msle = mean_squared_log_error(y_test_true, y_test_pred)
    error_metrics.append('MSLE')
    values.append(test_msle)
except ValueError:
    test_msle = None  # Handle cases where MSLE cannot be calculated

# Plot error metrics
plt.figure(figsize=(10, 6))
bars = plt.bar(error_metrics, values, color=['blue', 'green', 'red', 'orange'][:len(error_metrics)])
plt.xlabel('Error Metric')
plt.ylabel('Value')
plt.title('Comparison of Error Metrics (Test Set)')

# Annotate values on top of the bars
for bar, value in zip(bars, values):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{value:.2f}', 
             ha='center', va='bottom', fontsize=10)
plt.savefig(f'plots/tensor/03_metrics/{file_name}_test_set_metrics.png')
plt.show()

# Plotting R-squared (R2) for the test set
plt.figure(figsize=(6, 6))

if test_r2 >= 0:
    plt.pie([test_r2, 100 - test_r2], 
            labels=['Explained Variance (R2)', 'Unexplained Variance'], 
            colors=['lightblue', 'lightgrey'], autopct='%1.1f%%')
else:
    plt.pie([100], labels=['Unexplained Variance'], colors=['lightgrey'], autopct='%1.1f%%')

plt.title('Test Set Explained Variance by R-squared (R2)')
plt.savefig(f'plots/tensor/03_metrics/{file_name}_test_set_r2.png')
plt.show()

In [ ]:
# Logging info
logging.info("Calculating metrics for the Test Set...")

# Initialize error metrics
error_metrics = ['MSE', 'MAE', 'RMSE']
values = [external_mse, external_mae, external_rmse]

# Try to calculate MSLE
try:
    external_msle = mean_squared_log_error(y_external_true, y_external_pred)
    error_metrics.append('MSLE')
    values.append(test_msle)
except ValueError:
    external_msle = None  # Handle cases where MSLE cannot be calculated

# Plot error metrics
plt.figure(figsize=(10, 6))
bars = plt.bar(error_metrics, values, color=['blue', 'green', 'red', 'orange'][:len(error_metrics)])
plt.xlabel('Error Metric')
plt.ylabel('Value')
plt.title('Comparison of Error Metrics (Test Set)')

# Annotate values on top of the bars
for bar, value in zip(bars, values):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{value:.2f}', 
             ha='center', va='bottom', fontsize=10)
plt.savefig(f'plots/tensor/03_metrics/{file_name}_external_set_metrics.png')
plt.show()

# Plotting R-squared (R2) for the test set
plt.figure(figsize=(6, 6))

if test_r2 >= 0:
    plt.pie([test_r2, 100 - test_r2], 
            labels=['Explained Variance (R2)', 'Unexplained Variance'], 
            colors=['lightblue', 'lightgrey'], autopct='%1.1f%%')
else:
    plt.pie([100], labels=['Unexplained Variance'], colors=['lightgrey'], autopct='%1.1f%%')

plt.title('Test Set Explained Variance by R-squared (R2)')
plt.savefig(f'plots/tensor/03_metrics/{file_name}_external_set_r2.png')
plt.show()

In [ ]:
# Test Set Plot
plt.figure(figsize=(8, 6))
plt.scatter(y_test_true, y_test_pred, color='blue', label='Prediction')

# Line for Perfect Prediction
perfect_line = np.linspace(y_test_true.min(), y_test_true.max(), 100)
plt.plot(perfect_line, perfect_line, color='red', linestyle='--', label='Perfect Prediction')

# Labels, legend, and grid
plt.xlabel('True LOS')
plt.ylabel('Predicted LOS')
plt.legend()
plt.grid(True)
plt.title('Predicted vs. True LOS (Test Set)')

# Save the plot as a PNG image
plt.savefig(f"plots/tensor/04_prediction_plot/01_true_vs_pred/{file_name}_test_plot.png", dpi=300, bbox_inches='tight')
plt.show()

# External Validation Set Plot
plt.figure(figsize=(8, 6))
plt.scatter(y_external_true, y_external_pred, color='blue', label='Prediction')

# Line for Perfect Prediction (y = x)
perfect_line_ext = np.linspace(y_external_true.min(), y_external_true.max(), 100)
plt.plot(perfect_line_ext, perfect_line_ext, color='red', linestyle='--', label='Perfect Prediction')

# Labels, legend, and grid
plt.xlabel('True LOS')
plt.ylabel('Predicted LOS')
plt.legend()
plt.grid(True)
plt.title('Predicted vs. True LOS (External Validation Set)')

# Save the plot as a PNG image
plt.savefig(f"plots/tensor/04_prediction_plot/01_true_vs_pred/{file_name}_external_plot.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Convert y_test to a 1D numpy array
#y_test_true = y_test_true.numpy().flatten()


# Calculate residuals
residuals = y_test_true - y_test_pred

# Plot residuals
plt.figure(figsize=(8, 6))
plt.scatter(y_test_true, residuals, color='blue', alpha=0.5, label="Residuals")
plt.axhline(y=0, color='red', linestyle='--', label="Zero Line")
plt.axhline(y=test_mae, color='green', linestyle='--', label=f"MAE = {test_mae:.2f}")
plt.axhline(y=-test_mae, color='green', linestyle='--')
plt.xlabel('True LOS')
plt.ylabel('Residuals (True - Predicted)')
plt.title('Residuals Plot with MAE Bounds')
plt.grid(True)

# Place the legend outside of the plot
plt.legend(loc="upper left", bbox_to_anchor=(1, 1))

# Save the plot as a PNG image
plt.savefig(f"plots/tensor/04_prediction_plot/02_residuals/{file_name}_residuals_plot.png", dpi=300, bbox_inches='tight')
plt.show()